# Profiling the `hpctoolkit_dataframe` itself

In [1]:
import datetime
import pathlib

import line_profiler

import hpctoolkit_dataframe
from hpctoolkit_dataframe import HPCtoolkitDataFrame

In [2]:
profiler = line_profiler.LineProfiler()
profiler.add_module(hpctoolkit_dataframe)
profiler.enable()

df1 = HPCtoolkitDataFrame(path=pathlib.Path('.', 'test', 'data', 'experiment1.xml'), max_depth=None)
df2 = HPCtoolkitDataFrame(path=pathlib.Path('.', 'test', 'data', 'experiment2.xml'), max_depth=None)

profiler.disable()

test/data/experiment2.xml: percentage column candidates CPUTIME (usec):['Mean (I)', 'Sum (I)'] not found, trying ['PAPI_MEM_WCY']:['Mean (I)', 'Sum (I)']


In [3]:
profiler.enable()

hot_path1 = df1.hot_path(threshold=0.01)
hot_path2 = df2.hot_path(threshold=0.01)

profiler.disable()

In [4]:
profiler.enable()

df1.compact
df2.compact

profiler.disable()

In [5]:
profiler.enable()

df1.flame_graph()
df2.flame_graph(column='PAPI_MEM_WCY:Mean (I)')

profiler.disable()

<Figure size 1600x1600 with 1 Axes>

<Figure size 1600x1600 with 1 Axes>

In [6]:
profiler.print_stats()

Timer unit: 1e-06 s

Total time: 4.35858 s
File: /home/mateusz/Projects/hpctoolkit_dataframe/hpctoolkit_dataframe/hpctoolkit_dataframe.py
Function: __init__ at line 154

Line #      Hits         Time  Per Hit   % Time  Line Contents
   154                                               def __init__(self, *args,
   155                                                            path: pathlib.Path = None, max_depth: t.Optional[int] = None, **kwargs):
   156      3811      14992.0      3.9      0.3          if path is None:
   157      3809      82964.0     21.8      1.9              super().__init__(*args, **kwargs)
   158      3809      12837.0      3.4      0.3              return
   159         2         34.0     17.0      0.0          self._db_path = path
   160                                           
   161         2     257708.0 128854.0      5.9          profile_data = _read_xml(path).find('./SecCallPathProfile')
   162         2         35.0     17.5      0.0          _LOG.info(

# Profiling ways to query a DataFrame (obsolete)

## 1

`DataFrame.loc[[str(...)]]` vs `DataFrame[DataFrame.col == ...]` vs `DataFrame[DataFrame['col'] == ...]`

assuming that `DataFrame` is indexed by stringified column `col`

In [7]:
%%timeit

df.loc[[str(())]]

579 µs ± 34.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
type(df.loc[[str(())]])

hpctoolkit_dataframe.HPCtoolkitDataFrame

In [9]:
%%timeit

df[df.location == ()]

544 µs ± 36.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
type(df[df.location == ()])

hpctoolkit_dataframe.HPCtoolkitDataFrame

In [11]:
%%timeit

df[df['location'] == ()]

529 µs ± 36.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 2

`Series.at[...]` vs `Series.get(...)` vs `Series[...]`

In [12]:
filtered = df.loc['()']

In [13]:
type(filtered)

pandas.core.series.Series

In [14]:
%%timeit

df.loc['()']

135 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
%%timeit

filtered.at['location']

7.71 µs ± 56.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
type(filtered.at['location'])

tuple

In [17]:
%%timeit

filtered.get('location')

9.82 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
type(filtered.get('location'))

tuple

In [19]:
%%timeit

filtered['location']

9.4 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
type(filtered['location'])

tuple

## 3

`DataFrame.at['index', 'col']` vs `DataFrame.get('col').item()` vs `DataFrame['col'].item()`

assuming that `Dataframe` contains only one row (indexed as `'index'`)

In [21]:
filtered = df.loc[['()']]

In [22]:
type(filtered)

hpctoolkit_dataframe.HPCtoolkitDataFrame

In [23]:
%%timeit

df.loc[['()']]

522 µs ± 5.37 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%%timeit

filtered.at['()', 'location']

3.69 µs ± 55.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [25]:
type(filtered.at['()', 'location'])

tuple

In [26]:
%%timeit

filtered.get('location').item()

2.61 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
type(filtered.get('location').item())

tuple

In [28]:
%%timeit

filtered['location'].item()

2.43 µs ± 34.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [29]:
type(filtered['location'].item())

tuple